In [1]:
from pathlib import Path

voxel_sizes = [5, 2, 2] # in microns

In [ ]:
import pooch
data_path = Path.cwd().parent / "serial_2p_subset"
# Use pooch to fetch data if it hasn't already been downloaded
dsb_data_url = "https://gin.g-node.org/BrainGlobe/demo-materials/raw/master/serial2p_subset.zip"
data_path = pooch.retrieve(dsb_data_url, known_hash="c31e0136d00024ba74085baefbfd174c8e325f97a942c803ee720bdc8f6e7a00", progressbar=True, processor=pooch.Unzip(extract_dir=data_path))
data_path = Path(data_path[0]).parent.parent
signal_images = data_path / "ch00"
background_images = data_path / "ch02"

 33%|███████████▏                      | 1.84G/5.59G [1:11:46<4:28:26, 233kB/s]

In [3]:
import glob
import os

from dask import array as da
from dask import delayed
from imlib.general.system import get_sorted_file_paths
from tifffile import TiffFile, imread

In [5]:
# Write a function to load data with dask
# Alternatively - `from cellfinder_core.tools.IO import read_with_dask`

lazy_imread = delayed(imread) 

def get_tiff_meta(path):
    with TiffFile(path) as tfile:
        nz = len(tfile.pages)
        if not nz:
            raise ValueError(f"tiff file {path} has no pages!")
        first_page = tfile.pages[0]

    return tfile.pages[0].shape, first_page.dtype


def read_with_dask(path):
    filenames = glob.glob(os.path.join(path, "*.tif"))
    shape, dtype = get_tiff_meta(filenames[0])
    lazy_arrays = [lazy_imread(fn) for fn in get_sorted_file_paths(filenames)]
    dask_arrays = [
        da.from_delayed(delayed_reader, shape=shape, dtype=dtype)
        for delayed_reader in lazy_arrays
    ]
    stack = da.stack(dask_arrays, axis=0)
    return stack

In [6]:
from cellfinder_core.main import main as cellfinder_run

signal_array = read_with_dask(signal_images)
background_array = read_with_dask(background_images)

print(signal_array)
print(background_array)

dask.array<stack, shape=(100, 3868, 5416), dtype=uint16, chunksize=(1, 3868, 5416), chunktype=numpy.ndarray>
dask.array<stack, shape=(100, 3868, 5416), dtype=uint16, chunksize=(1, 3868, 5416), chunktype=numpy.ndarray>


In [7]:
import napari
viewer = napari.view_image(background_array)
viewer.add_image(signal_array)

<Image layer 'signal_array' at 0x281a1f850>

In [8]:
detected_cells = cellfinder_run(signal_array,background_array,voxel_sizes)

Processing planes: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:46<00:00,  1.07s/it]
/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/numba/typed/typeddict.py:39: NumbaTypeSafetyWarning: unsafe cast from int64 to uint64. Precision may be lost.
  return d[key]


Detection complete - all planes done in : 0:03:28.140525
237/237 [==============================] - 428s 2s/step


Loading brainreg directory
Found additional downsampled image: downsampled_standard_2.tiff, adding to viewer


Traceback (most recent call last):
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/app/backends/_qt.py", line 532, in mouseMoveEvent
    self._vispy_mouse_move(
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/app/base.py", line 216, in _vispy_mouse_move
    ev = self._vispy_canvas.events.mouse_move(**kwargs)
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/napari/_qt/qt_

Adding a new region



...

Traceback (most recent call last):
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/app/backends/_qt.py", line 532, in mouseMoveEvent
    self._vispy_mouse_move(
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/app/base.py", line 216, in _vispy_mouse_move
    ev = self._vispy_canvas.events.mouse_move(**kwargs)
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/napari/_qt/qt_

Preventing analysis as user chose 'Cancel'


Traceback (most recent call last):
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/app/backends/_qt.py", line 532, in mouseMoveEvent
    self._vispy_mouse_move(
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/app/base.py", line 216, in _vispy_mouse_move
    ev = self._vispy_canvas.events.mouse_move(**kwargs)
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/napari/_qt/qt_

Running region analysis
Calculating region volume distribution
Saving summary volumes to: /Users/adamtyson/analysis/CAA_1110263_allen_10um/manual_segmentation/standard_space/regions


Traceback (most recent call last):
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/app/backends/_qt.py", line 532, in mouseMoveEvent
    self._vispy_mouse_move(
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/app/base.py", line 216, in _vispy_mouse_move
    ev = self._vispy_canvas.events.mouse_move(**kwargs)
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/Users/adamtyson/miniconda3/envs/image-analysis/lib/python3.10/site-packages/napari/_qt/qt_

In [22]:
len(detected_cells)

1542

In [ ]:
from cellfinder_napari.utils import add_layers as add_cell_layers
add_cell_layers(detected_cells, viewer)